In [105]:
import os
import pandas as pd
import numpy as np

base_dir = "../Data"
base_csv_path = os.path.join(base_dir, 'base.csv')
assert os.path.exists(base_csv_path), f"base {base_csv_path} does not exist"

output_csv_path = os.path.join(base_dir, 'm4_imputed.csv')
df = pd.read_csv(base_csv_path)

/var/folders/k_/917q0lsn2g199rm9y4s8jy5h0000gn/T/ipykernel_7440/4113478112.py:10: DtypeWarning: Columns (2,64,91,94,97,106,108,109,110,111,112,114,116,119,120,122,127,139,142,143,144,145,152,153,154,155,156,157,159,160,161,163,164,165,169,170,171,172,175,178,179,180,187,189,215,216,217,218,219,317,318) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(base_csv_path)


## Feature selection

In [106]:
# Replace "ethnicity" has been replaced with ethcat variable
desired_columns = ['thoracic_dgn', 'num_prev_tx', 'tah', 'vas', 'onvent', 'icu', 'inotropic', 'gender', 'abo',
                   'wgt_kg_tcr', 'hgt_cm_tcr', 'education', 'ecmo_tcr', 'iabp_tcr', 'inotropes_tcr', 'func_stat_tcr',
                   'diab', 'dial_ty_tcr', 'cereb_vasc', 'malig_tcr', 'most_rcnt_creat', 'tot_serum_album',
                   'hemo_co_tcr', 'cig_use', 'prior_card_surg_tcr', 'histry_cig_old', 'init_stat', 'init_creat',
                   'init_age', 'ethcat', 'init_hgt_cm_calc', 'init_wgt_kg_calc', 'ventilator_tcr', 'lvad_at_listing',
                   'rvad_at_listing', 'work_income_tcr', 'academic_level_tcr', 'tx_date', 'init_date']

df = df[desired_columns]

In [107]:
# Remove all init_age under 18
pre_len = len(df.index)
df = df[df['init_age'] >= 18]
post_len = len(df.index)
print(
    f"Removed patients under 18: {pre_len - post_len}/{pre_len} ({(pre_len - post_len) / pre_len * 100:.2f}% removed)")

Removed patients under 18: 16205/120264 (13.47% removed)


## Encoding

In [108]:
print(f'Number of rows with value 6 in ethcat before replacing values: {len(df[df["ethcat"] == 6])}')

#1=1 (White), 2=2 (Black), 4=4 (Hispanic), 5=5 (Asian),6=6 (tidigare Amer Ind/Alaskan, kodas nu om till Other) 7=6 (tidigare Native Hawaiian, nu Other) 9=6 (tidigare Multiracial, nu Other)
df['ethcat'] = df['ethcat'].replace([7, 9, 998], 6)

print(f'Number of rows with value 6 in ethcat after replacing values: {len(df[df["ethcat"] == 6])}')

Number of rows with value 6 in ethcat before replacing values: 357
Number of rows with value 6 in ethcat after replacing values: 1336


In [109]:
# Encode education

df['education'] = df['education'].replace([2, 3], 1)  # High school or less
df['education'] = df['education'].replace([4], 2)  # Some college
df['education'] = df['education'].replace([5, 6], 3)  # College or graduate
df['education'] = df['education'].replace([996, 998], np.nan)

In [110]:
# Encode init_stat: label define status 1 "Status 1A" 2 "Status 1B" 3 "Status 2" 4 "Temp inactive"

df["init_stat"] = df["init_stat"].replace([2010, 2110, 2120, 2130, 2090], 1)
df["init_stat"] = df["init_stat"].replace([2020, 2140], 2)
df["init_stat"] = df["init_stat"].replace([2030, 2150, 2160], 3)
df["init_stat"] = df["init_stat"].replace([2999], 4)

In [111]:
import numpy as np

# Encode thoracic_dgn: label define diag 1 "Cardiomyopathy" 2 "CAD" 3 "Valvular heart disease" 4 "Graft failure" 5 "Congenital" 7 "Other"
df["thoracic_dgn"] = df["thoracic_dgn"].replace(range(1000, 1099 + 1), 1)
df["thoracic_dgn"] = df["thoracic_dgn"].replace([1201], 1)
df["thoracic_dgn"] = df["thoracic_dgn"].replace([1007, 1200], 2)
df["thoracic_dgn"] = df["thoracic_dgn"].replace([1202], 3)
df["thoracic_dgn"] = df["thoracic_dgn"].replace(range(1100, 1199 + 1), 4)
df["thoracic_dgn"] = df["thoracic_dgn"].replace(range(1203, 1207 + 1), 5)
df["thoracic_dgn"] = df["thoracic_dgn"].replace([1208, 1209, 999, 1497, 1498], 7)
df["thoracic_dgn"] = df["thoracic_dgn"].replace([1999], np.nan)

In [112]:
df["func_stat_tcr"] = df["func_stat_tcr"].replace([2010, 2020, 2030, 2040, 2050, 4010, 4020, 4030, 4040, 4050], 3)
df["func_stat_tcr"] = df["func_stat_tcr"].replace([2060, 2070, 2080, 4060, 4070, 4080], 2)
df["func_stat_tcr"] = df["func_stat_tcr"].replace([2090, 2100, 4090, 4100], 1)
df["func_stat_tcr"] = df["func_stat_tcr"].replace([996, 998], np.nan)

In [113]:
df["abo"] = df["abo"].replace(["A", "A1", "A2"], 1)
df["abo"] = df["abo"].replace(["B"], 2)
df["abo"] = df["abo"].replace(["AB", "A1B", "A2B"], 3)
df["abo"] = df["abo"].replace(["O"], 4)

In [114]:
df["diab"] = df["diab"].replace([996, 998], np.nan)
df["dial_ty_tcr"] = df["dial_ty_tcr"].replace([996, 998, 999], np.nan)
df["academic_level_tcr"] = df["academic_level_tcr"].replace([996, 998], np.nan)

## Cleaning

In [115]:
# Remove rows with no transplant date
pre_len = len(df.index)
df = df[df['tx_date'].notna()]
post_len = len(df.index)

print(f"Filtered to only transplanted patients: {post_len}/{pre_len} ({post_len / pre_len * 100:.2f}% remaining)")

# Compute the time to transplant (waitlist time -> wl_time)
df['wl_time'] = (
        pd.to_datetime(df['tx_date'], format='%d%b%Y') - pd.to_datetime(df['init_date'], format='%d%b%Y')).dt.days
df = df.drop(columns=['tx_date', 'init_date'])
print(f"Mean waitlist time: {df['wl_time'].median():.2f} days")


Filtered to only transplanted patients: 66880/104059 (64.27% remaining)
Mean waitlist time: 87.00 days


In [116]:
# Remove columns where all values are missing
pre_len = len(df.columns)
df = df.dropna(axis=1, how='all')
post_len = len(df.columns)
print(
    f"Removed columns with all missing values: {pre_len - post_len}/{pre_len} ({(pre_len - post_len) / pre_len * 100:.2f}% removed)")

# Remove columns where there is no variance
pre_len = len(df.columns)
df = df.loc[:, df.nunique() != 1]
post_len = len(df.columns)
print(
    f"Removed columns with no variance: {pre_len - post_len}/{pre_len} ({(pre_len - post_len) / pre_len * 100:.2f}% removed)")


Removed columns with all missing values: 1/38 (2.63% removed)
Removed columns with no variance: 2/37 (5.41% removed)


In [117]:
# Remove weights under 30kg and over 250kg
pre_len = len(df.index)
df = df[df['init_wgt_kg_calc'] >= 30]
df = df[df['wgt_kg_tcr'] >= 30]
df = df[df['init_wgt_kg_calc'] <= 250]
df = df[df['wgt_kg_tcr'] <= 250]
post_len = len(df.index)
print(
    f"Removed patients with weight under 30kg or over 250kg: {pre_len - post_len}/{pre_len} ({(pre_len - post_len) / pre_len * 100:.2f}% removed)")

# Remove heights under 140cm and over 210cm
pre_len = len(df.index)
df = df[df['init_hgt_cm_calc'] >= 140]
df = df[df['hgt_cm_tcr'] >= 140]
df = df[df['init_hgt_cm_calc'] <= 210]
df = df[df['hgt_cm_tcr'] <= 210]
post_len = len(df.index)
print(
    f"Removed patients with height under 140cm or over 210cm: {pre_len - post_len}/{pre_len} ({(pre_len - post_len) / pre_len * 100:.2f}% removed)")

# Remove patients with creatinine under 0.3 and over 20 - keep missing values
pre_len = len(df.index)
df = df[df['most_rcnt_creat'] >= 0.3]
df = df[df['most_rcnt_creat'] <= 20]
post_len = len(df.index)
print(
    f"Removed patients with creatinine under 0.3 or over 20: {pre_len - post_len}/{pre_len} ({(pre_len - post_len) / pre_len * 100:.2f}% removed)")

# Remove patients with albumin under 2 and over 7.5
pre_len = len(df.index)
df = df[df['tot_serum_album'] >= 2]
df = df[df['tot_serum_album'] <= 7.5]
post_len = len(df.index)
print(
    f"Removed patients with albumin under 2 or over 7.5: {pre_len - post_len}/{pre_len} ({(pre_len - post_len) / pre_len * 100:.2f}% removed)")

# Remove cardiac output under 1 and over 20
pre_len = len(df.index)
df = df[df['hemo_co_tcr'] >= 1]
df = df[df['hemo_co_tcr'] <= 15]
post_len = len(df.index)
print(
    f"Removed patients with cardiac output under 1 or over 20: {pre_len - post_len}/{pre_len} ({(pre_len - post_len) / pre_len * 100:.2f}% removed)")

Removed patients with weight under 30kg or over 250kg: 842/66880 (1.26% removed)
Removed patients with height under 140cm or over 210cm: 436/66038 (0.66% removed)
Removed patients with creatinine under 0.3 or over 20: 22807/65602 (34.77% removed)
Removed patients with albumin under 2 or over 7.5: 17256/42795 (40.32% removed)
Removed patients with cardiac output under 1 or over 20: 2055/25539 (8.05% removed)


In [118]:
#Creat
output_csv_path_cleaned = os.path.join(base_dir, 'base_cleaned.csv')
df.to_csv(output_csv_path_cleaned, index=False)

In [119]:
# Find the categorical columns using nunique
categorical_columns = df.nunique()[df.nunique() < 20].index.tolist()
categorical_column_indexes = [df.columns.get_loc(c) for c in categorical_columns if c in df]

# Print the number of categorical columns
print(f"Number of categorical columns: {len(categorical_columns)}")
print(f"Categorical columns: {categorical_columns}")

# print unique values in each categorical column
for col in categorical_columns:
    print(f"{col}: {df[col].unique()}")

Number of categorical columns: 26
Categorical columns: ['thoracic_dgn', 'num_prev_tx', 'tah', 'vas', 'onvent', 'icu', 'inotropic', 'gender', 'abo', 'education', 'ecmo_tcr', 'iabp_tcr', 'inotropes_tcr', 'func_stat_tcr', 'diab', 'dial_ty_tcr', 'cereb_vasc', 'malig_tcr', 'cig_use', 'prior_card_surg_tcr', 'histry_cig_old', 'init_stat', 'ethcat', 'ventilator_tcr', 'work_income_tcr', 'academic_level_tcr']
thoracic_dgn: [ 1.  2.  4.  3.  7.  5. nan]
num_prev_tx: [ 0  1  2  3  6 10  5  4]
tah: [nan 'N']
vas: [nan 'N' 'Y']
onvent: [nan 'N' 'Y']
icu: [nan 'Y' 'N']
inotropic: [nan 'Y' 'N']
gender: ['M' 'F']
abo: [4 2 3 1]
education: [ 3.  2.  1. nan]
ecmo_tcr: [0 1]
iabp_tcr: [0 1]
inotropes_tcr: [1 0]
func_stat_tcr: [ 3.  2. nan  1.]
diab: [ 1.  3.  5.  2. nan  4.]
dial_ty_tcr: [ 1.  2. nan  3.]
cereb_vasc: ['N' 'Y' 'U']
malig_tcr: ['N' 'Y' 'U']
cig_use: ['Y' nan 'N']
prior_card_surg_tcr: ['Y' 'U' 'N' nan]
histry_cig_old: [nan 'Y' 'U' 'N']
init_stat: [ 3.  2.  1.  4. nan]
ethcat: [1 2 4 6 5]
ven

In [120]:
# print all categorical columns and their amount of unique values
for col in categorical_columns:
    print(f"{col}: {len(df[col].unique())}")

# print all min and max values for each column
numerical_columns = [col for col in df.columns if col not in categorical_columns]
for col in numerical_columns:
    amount_missing = df[col].isna().sum()
    print(f"{col}: {df[col].min()} - {df[col].max()} ({amount_missing} missing)")

thoracic_dgn: 7
num_prev_tx: 8
tah: 2
vas: 3
onvent: 3
icu: 3
inotropic: 3
gender: 2
abo: 4
education: 4
ecmo_tcr: 2
iabp_tcr: 2
inotropes_tcr: 2
func_stat_tcr: 4
diab: 6
dial_ty_tcr: 4
cereb_vasc: 3
malig_tcr: 3
cig_use: 3
prior_card_surg_tcr: 4
histry_cig_old: 4
init_stat: 5
ethcat: 5
ventilator_tcr: 2
work_income_tcr: 4
academic_level_tcr: 4
wgt_kg_tcr: 31.752 - 177.8082 (0 missing)
hgt_cm_tcr: 141.0 - 208.28 (0 missing)
most_rcnt_creat: 0.3 - 19.5 (0 missing)
tot_serum_album: 2.0 - 7.5 (0 missing)
hemo_co_tcr: 1.0 - 15.0 (0 missing)
init_age: 18.0 - 77.0 (0 missing)
init_hgt_cm_calc: 141.0 - 208.28 (0 missing)
init_wgt_kg_calc: 31.752 - 177.8082 (0 missing)
wl_time: 0 - 5152 (0 missing)


In [121]:
from sklearn.preprocessing import OrdinalEncoder
from missingpy import MissForest

ordinal_encoder = OrdinalEncoder()
# Encode the categorical columns
df[categorical_columns] = ordinal_encoder.fit_transform(df[categorical_columns])

imputer = MissForest()

wl_time = df.pop('wl_time')

df = pd.DataFrame(imputer.fit_transform(df, cat_vars=categorical_column_indexes), columns=df.columns)

df['wl_time'] = wl_time.values

df[categorical_columns] = ordinal_encoder.inverse_transform(df[categorical_columns])

/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/missingpy/missforest.py:475: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  col_modes = mode(
/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: F

Iteration: 0


/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sq

Iteration: 1


/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/homebrew/anaconda3/envs/edap01/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sq

Iteration: 2


In [122]:
df = pd.get_dummies(df, columns=categorical_columns)
df.to_csv(output_csv_path, index=False)

/var/folders/k_/917q0lsn2g199rm9y4s8jy5h0000gn/T/ipykernel_7440/1227575835.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df, columns=categorical_columns)
/var/folders/k_/917q0lsn2g199rm9y4s8jy5h0000gn/T/ipykernel_7440/1227575835.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df, columns=categorical_columns)
/var/folders/k_/917q0lsn2g199rm9y4s8jy5h0000gn/T/ipykernel_7440/1227575835.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df, columns=categorical_columns)
/var/folders/k_/917q0lsn2g199rm9y4s8jy5h0000gn/T/ipykernel_7440/1227575835.py:1: FutureWarning: In a future version, the Index constructor wil